In [1]:
from helpers import read_wiki_sents

sents = read_wiki_sents()
sents[:5]

['Gemeinsam mit Inspektor Kemp, einem ehemaligen Schüler des aus anderen Christie-Romanen bekannten Inspektor Battle, untersucht er nun die Todesfälle.',
 'Im Juni 2020 wurde die modernisierte Version M-84AS1 vorgestellt.',
 'In Großbritannien erfreute sich Metternich ungeachtet des deutsch-kritischen Klimas großen Ansehens.',
 'Gertrud von le Fort teilt weder irgendeine Jahreszahl noch den Namen des Königs mit.',
 'In Rio unterstützt EBX Sport-, Bewirtungs-, Gastronomie-, Gesundheits- und Schönheitsinitiativen.']

In [2]:
from datasets import load_dataset
train = list(load_dataset("diversifix/inclusive_words")["train"])
train[:3]

Using custom data configuration diversifix--inclusive_words-cbfc100d5496cbf7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /Users/david/.cache/huggingface/datasets/csv/diversifix--inclusive_words-cbfc100d5496cbf7/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

[{'exclusive': 'Abbrecher',
  'inclusive': 'Abbrecherin',
  'applicable': 'always',
  'gender_of_inclusive': 'female',
  'source': 'dereko'},
 {'exclusive': 'Abbrecherquote',
  'inclusive': 'Abbruchquote',
  'applicable': 'always',
  'gender_of_inclusive': 'neutral',
  'source': 'geschicktgendern'},
 {'exclusive': 'Abenteurer',
  'inclusive': 'abenteuerliebende Person',
  'applicable': 'always',
  'gender_of_inclusive': 'neutral',
  'source': 'geschicktgendern'}]

In [3]:
data = dict()
for row in train:
    if not row["exclusive"] in data.keys():
        data[row["exclusive"]] = []
    data[row["exclusive"]].append(row)

In [4]:
import spacy
nlp = spacy.load("de_core_news_sm", disable=["ner", "attribute_ruler", "parser"])
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x282f8bdc0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x282f8bca0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x282f8b820>),
 ('lemmatizer',
  <spacy.pipeline.edit_tree_lemmatizer.EditTreeLemmatizer at 0x282b08ca0>)]

In [5]:
docs = list(nlp.pipe(sents[:4000], batch_size=1000, n_process=4))

In [6]:
import random
random.seed(93020)

people_sents = []
for doc in docs:
    matches = [t for t in doc if t.pos_ == "NOUN" and t.lemma_ in data.keys()]
    if len(matches) == 0:
        continue
    random.shuffle(matches)
    t = matches[0]
    alternatives = data[t.lemma_]
    if t.morph.get("Number") == ["Sing"]:
        alternatives = [a for a in alternatives if a["applicable"] in ["in_singular", "always"]]
    if len(alternatives) == 0:
        continue
    random.shuffle(alternatives)
    alt = alternatives[0]
    post = " oder " + t.lemma_ if alt["gender_of_inclusive"] == "female" and random.random() > 0.5 else ""
    inclusive = alt["inclusive"] + post
    people_sents.append((doc.text, t.lemma_, inclusive))
(len(people_sents), people_sents[:3])

(778,
 [('Gemeinsam mit Inspektor Kemp, einem ehemaligen Schüler des aus anderen Christie-Romanen bekannten Inspektor Battle, untersucht er nun die Todesfälle.',
   'Schüler',
   'Schulkind'),
  ('Gertrud von le Fort teilt weder irgendeine Jahreszahl noch den Namen des Königs mit.',
   'König',
   'Königin oder König'),
  ('Die Applizierung der Drehung ins Handgelenk kann nun optimal mit der zum Angreifer einwärts gedrehten Schwertklinge unterstützt werden.',
   'Angreifer',
   'angreifende Person')])

In [7]:
from helpers import chunks

batches = []
for chunk in chunks(people_sents, 8):
    inputs = ["""(1)\nOriginal: "Die Schüler kamen zu spät."\nMit Ersetzung: "Die Schülerinnen und Schüler kamen zu spät."\n\n(2)\nOriginal: "Sie werden dem neuen Kanzler gratulieren."\nMit Ersetzung: "Sie werden der neuen Kanzlerin oder dem neuen Kanzler gratulieren."\n"""]
    instructions = ["""Führe die folgenden Ersetzungen durch. Verändere den Satz und die Ersatzwörter dazu gegebenenfalls grammatisch, sodass ein grammatisch korrekter und flüssiger Satz entsteht.\n\n(1) Ersetze "Schüler" durch eine entsprechend angepasste Form von "Schülerin oder Schüler".\n(2) Ersetze "Kanzler" durch eine entsprechend angepasste Form von "Kanzlerin oder Kanzler"."""]
    for i, (sent, a, b) in enumerate(chunk, 3):
        inputs.append(f"""({i})\nOriginal: "{sent}"\nMit Ersetzung: ___\n""")
        instructions.append(f"""({i}) Ersetze "{a}" durch eine entsprechend angepasste Form von "{b}".""")
    batches.append((chunk, "\n".join(inputs), "\n".join(instructions)))
print(batches[0][1])
print(batches[0][2])

(1)
Original: "Die Schüler kamen zu spät."
Mit Ersetzung: "Die Schülerinnen und Schüler kamen zu spät."

(2)
Original: "Sie werden dem neuen Kanzler gratulieren."
Mit Ersetzung: "Sie werden der neuen Kanzlerin oder dem neuen Kanzler gratulieren."

(3)
Original: "Gemeinsam mit Inspektor Kemp, einem ehemaligen Schüler des aus anderen Christie-Romanen bekannten Inspektor Battle, untersucht er nun die Todesfälle."
Mit Ersetzung: ___

(4)
Original: "Gertrud von le Fort teilt weder irgendeine Jahreszahl noch den Namen des Königs mit."
Mit Ersetzung: ___

(5)
Original: "Die Applizierung der Drehung ins Handgelenk kann nun optimal mit der zum Angreifer einwärts gedrehten Schwertklinge unterstützt werden."
Mit Ersetzung: ___

(6)
Original: "Außerdem habe sie sich von ihrer Mutter zu sehr bevormundet gefühlt, und sich daran gestört, dass es dieser so überaus wichtig sei, was die Leute über sie denken."
Mit Ersetzung: ___

(7)
Original: "Der Sohn eines berühmten Musikers erzählt."
Mit Ersetzung: 

In [8]:
%load_ext dotenv
%dotenv

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


In [9]:
from joblib import Memory
memory = Memory(".cache", verbose=0)

@memory.cache
def get_replacements(input, instruction):
    response = openai.Edit.create(
        engine="text-davinci-edit-001",
        input=input,
        instruction=instruction,
        temperature=0,
        top_p=1
    )
    return response["choices"][0]["text"]

In [10]:
from itertools import chain
import re

from joblib import Parallel, delayed
from tqdm.notebook import tqdm

def get_unfiltered_training_data(batch):
    chunk, input, instruction = batch
    output = get_replacements(input, instruction)
    replacements = re.findall(r"Mit Ersetzung: \"(.*)\"", output)[2:]
    return [(sent, a, b, rep) for (sent, a, b), rep in zip(chunk, replacements)]

utd = Parallel(n_jobs=4)(delayed(get_unfiltered_training_data)(batch) for batch in tqdm(batches[:12]))
unfiltered_training_data = list(chain(*utd))
    

  0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
import requests

training_data = []

for sent, a, b, rep in unfiltered_training_data:
    r = requests.post(
        "http://localhost:8081/v2/check",
        data={"text": rep, "language": "de-DE", "enabledCategories": "PUNCTUATION,CASING,COLLOCATIONS,CONFUSED_WORDS,CREATIVE_WRITING,GRAMMAR,MISC,MISUSED_TERMS_EU_PUBLICATIONS,NONSTANDARD_PHRASES,REDUNDANCY,SEMANTICS,TEXT_ANALYSIS,STYLE", "disabledCategories": "TYPOS,TYPOGRAPHY"},
        headers={"Content-Type": "application/x-www-form-urlencoded"},
    )
    matches = r.json()["matches"]
    if len(matches) == 0:
        training_data.append(dict(x=sent, a=a, b=b, y=rep))
(len(unfiltered_training_data), len(training_data))

(89, 71)

In [12]:
import json
with open("../data/training_data_gender.json", "w") as f:
    json.dump(training_data, f, ensure_ascii=False, indent=2)